In [1]:
import torch
import torch.nn as nn
from tqdm import tqdm

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1)) # 单隐藏层
X = torch.rand(2, 4)
net(X)

tensor([[0.1967],
        [0.1589]], grad_fn=<AddmmBackward0>)

In [2]:
print(net.state_dict())

OrderedDict([('0.weight', tensor([[ 0.4847, -0.4526,  0.0317, -0.2925],
        [ 0.2778,  0.3856,  0.4092, -0.1835],
        [ 0.3982, -0.4746,  0.2345, -0.1104],
        [-0.2298, -0.3835,  0.2261, -0.1941],
        [-0.4543,  0.1339,  0.2965,  0.2892],
        [-0.4103, -0.4889,  0.1125, -0.0478],
        [-0.0757,  0.4223, -0.4418,  0.0792],
        [ 0.1668, -0.1568,  0.3403, -0.0880]])), ('0.bias', tensor([-0.0674, -0.1886, -0.4037,  0.3223, -0.4523,  0.4107, -0.4542,  0.0603])), ('2.weight', tensor([[-0.0631,  0.1582, -0.2782,  0.3282, -0.2670, -0.3435,  0.2095, -0.3272]])), ('2.bias', tensor([0.1773]))])


In [3]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.1773], requires_grad=True)
tensor([0.1773])


In [4]:
net[2].weight.grad == None

True

In [5]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [6]:
net.state_dict()['2.bias'].data

tensor([0.1773])

In [7]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in tqdm(range(4)):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

100%|███████████████████████████████████████████| 4/4 [00:00<00:00, 4260.34it/s]


tensor([[-0.2900],
        [-0.2900]], grad_fn=<AddmmBackward0>)

In [8]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [9]:
def init_params(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, 0, 0.01)
        nn.init.zeros_(m.bias)
        
net.apply(init_params)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0074,  0.0063, -0.0079,  0.0104]), tensor(0.))

In [10]:
def init_constant(m):
    if isinstance(m, nn.Linear):
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
        
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [11]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data[0])

tensor([ 0.0009, -0.6261, -0.5045, -0.6563])
tensor([42., 42., 42., 42., 42., 42., 42., 42.])


In [12]:
def my_init(m):
    if isinstance(m, nn.Linear):
        print(
            "Init",
            *[(name, param.shape) for name, param in m.named_parameters()][0]
        )
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5 # 将小于5的元素置0

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000, -0.0000, -0.0000,  0.0000],
        [-9.6351, -0.0000, -7.5010,  9.3118]], grad_fn=<SliceBackward0>)

In [13]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.,  1.,  1.,  1.])

In [14]:
shared = nn.Linear(8, 8)
net = nn.Sequential(
    nn.Linear(4, 8), 
    nn.ReLU(),
    shared,
    nn.ReLU(),
    shared,
    nn.ReLU(),
    nn.Linear(8, 1)
)
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
